---
title: "Multinomial Logit Model"
format: html
author: "Tweety"
date: 2025-05-20
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
---




This assignment expores two methods for estimating the MNL model: (1) via Maximum Likelihood, and (2) via a Bayesian approach using a Metropolis-Hastings MCMC algorithm. 


## 1. Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 3 products, then either $y=3$ or $y=(0,0,1)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, brand, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 3 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^3e^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$



## 2. Simulate Conjoint Data

We will simulate data from a conjoint experiment about video content streaming services. We elect to simulate 100 respondents, each completing 10 choice tasks, where they choose from three alternatives per task. For simplicity, there is not a "no choice" option; each simulated respondent must select one of the 3 alternatives. 

Each alternative is a hypothetical streaming offer consistent of three attributes: (1) brand is either Netflix, Amazon Prime, or Hulu; (2) ads can either be part of the experience, or it can be ad-free, and (3) price per month ranges from \$4 to \$32 in increments of \$4.

The part-worths (ie, preference weights or beta parameters) for the attribute levels will be 1.0 for Netflix, 0.5 for Amazon Prime (with 0 for Hulu as the reference brand); -0.8 for included adverstisements (0 for ad-free); and -0.1*price so that utility to consumer $i$ for hypothethical streaming service $j$ is 

$$
u_{ij} = (1 \times Netflix_j) + (0.5 \times Prime_j) + (-0.8*Ads_j) - 0.1\times Price_j + \varepsilon_{ij}
$$

where the variables are binary indicators and $\varepsilon$ is Type 1 Extreme Value (ie, Gumble) distributed.

The following code provides the simulation of the conjoint data.

:::: {.callout-note collapse="true"}
## Simulating the Data


In [ ]:
#| echo: true
#| warning: false
#| code-overflow: wrap

import numpy as np
import pandas as pd
from itertools import product

np.random.seed(123)

# Attribute Levels 
brand_levels = ["N", "P", "H"]        
ad_levels = ["Yes", "No"]             
price_levels = np.arange(4, 33, 4)    

# All Possible Profiles 
profiles = pd.DataFrame(
    list(product(brand_levels, ad_levels, price_levels)),
    columns=["brand", "ad", "price"]
)

# Part-Worth Utilities 
b_util = {"N": 1.0, "P": 0.5, "H": 0.0}        
a_util = {"Yes": -0.8, "No": 0.0}           
price_util = lambda p: -0.1 * p               

# Simulation Parameters 
n_peeps = 100     
n_tasks = 10      
n_alts = 3      

# Simulate One Respondent’s Data 
def simulate_one(respondent_id: int) -> pd.DataFrame:
    tasks = []

    for task_no in range(1, n_tasks + 1):
        dat = profiles.sample(n=n_alts).copy()
        dat.insert(0, "task", task_no)
        dat.insert(0, "resp", respondent_id)

        # Deterministic utility
        dat["v"] = (
            dat["brand"].map(b_util)
            + dat["ad"].map(a_util)
            + price_util(dat["price"])
        )

        # Gumbel-distributed noise
        e = -np.log(-np.log(np.random.uniform(size=n_alts)))
        dat["u"] = dat["v"] + e

        # Determine choice
        dat["choice"] = (dat["u"] == dat["u"].max()).astype(int)

        tasks.append(dat)

    return pd.concat(tasks, ignore_index=True)

# Simulate All Respondents
conjoint_data = pd.concat(
    [simulate_one(i) for i in range(1, n_peeps + 1)],
    ignore_index=True
)

# Keep Only Observable Data 
conjoint_data = conjoint_data[["resp", "task", "brand", "ad", "price", "choice"]]

# Display Result Summary 
rows, cols = conjoint_data.shape
print(f"The simulated dataset contains {rows} rows and {cols} columns "
      f"({n_peeps} respondents × {n_tasks} tasks × {n_alts} alternatives).")

# Peek at Data 
print(conjoint_data.head())

:::: 


## 3. Preparing the Data for Estimation

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). The fact that each task for each respondent has the same number of alternatives (3) helps.  In addition, we need to convert the categorical variables for brand and ads into binary variables.


In [ ]:
# | echo: false
# Load the simulated conjoint dataset
df = pd.read_csv("conjoint_data.csv")

# Create dummy indicators for brand (baseline: Hulu) and ad (baseline: No-ad)
df["brand_N"] = (df["brand"] == "N").astype(int)  
df["brand_P"] = (df["brand"] == "P").astype(int)  
df["ad_yes"] = (df["ad"] == "Yes").astype(int) 

# Create a unique task identifier for each choice task (for grouping alternatives)
df["task_id"] = (df["resp"] - 1) * 10 + (df["task"] - 1)

print(df.head())

Each task_id represents a choice task with three alternatives. For example, task_id = 0 (resp = 1, task = 1) includes three options from brands N, H, and P, all with ads and different prices. The first row (brand = N, price = 28) has choice = 1, indicating it was selected. Dummy variables for brand and ad presence are created, with Hulu and ad-free as baselines. The data is now formatted for model estimation.

## 4. Estimation via Maximum Likelihood

:::: {.callout-note collapse="true"}
## Simulating the Data


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

X = df[["brand_N", "brand_P", "ad_yes", "price"]].values
y = df["choice"].values
groups = df["task_id"].values

# MNL log-likelihood
def mnl_log_likelihood(beta, X, y, task_ids):
    v = X @ beta
    exp_v = np.exp(v)
    df_v = pd.DataFrame({"task_id": task_ids, "exp_v": exp_v})
    denom = df_v.groupby("task_id")["exp_v"].transform("sum").values
    log_probs = v - np.log(denom)
    return -np.sum(y * log_probs)

# MLE estimation
beta_init = np.zeros(X.shape[1])
result = minimize(
    mnl_log_likelihood,
    beta_init,
    args=(X, y, groups),
    method="BFGS"
)
beta_hat = result.x
se = np.sqrt(np.diag(result.hess_inv))
z = 1.96
ci_lower = beta_hat - z * se
ci_upper = beta_hat + z * se

# MLE results
mle_summary = pd.DataFrame({
    "MLE Coefficient": beta_hat,
    "Std. Error": se,
    "95% CI Lower": ci_lower,
    "95% CI Upper": ci_upper
}, index=["brand_N", "brand_P", "ad_yes", "price"])

:::: 


In [ ]:
# | echo: false
mle_summary

The MLE estimates are:

- **brand_N**: 0.94  
- **brand_P**: 0.50  
- **ad_yes**: -0.73  
- **price**: -0.099  

Standard errors range from 0.006 to 0.12, and all 95% confidence intervals exclude zero. The model converged successfully and parameters were estimated with high precision.


## 5. Estimation via Bayesian Methods

::::{.callout-note collapse="true"}
## Simulating the Data


In [ ]:
proposal_sd = np.array([0.05, 0.05, 0.05, 0.005])

# Log-prior: N(0, 5^2) for binary vars, N(0, 1^2) for price
def log_prior(beta):
    return -0.5 * (np.sum(beta[:3]**2 / 25) + beta[3]**2)

# Log-posterior: log-likelihood + log-prior
def log_posterior(beta, X, y, task_ids):
    v = X @ beta
    exp_v = np.exp(v)
    df_v = pd.DataFrame({"task_id": task_ids, "exp_v": exp_v, "v": v, "y": y})
    denom = df_v.groupby("task_id")["exp_v"].transform("sum").values
    log_probs = v - np.log(denom)
    ll = np.sum(y * log_probs)
    return ll + log_prior(beta)

# MCMC sampler
def metropolis_sampler(X, y, task_ids, n_iter=11000, burn_in=1000):
    n_params = X.shape[1]
    samples = np.zeros((n_iter, n_params))
    beta_current = np.zeros(n_params)
    log_post_current = log_posterior(beta_current, X, y, task_ids)

    for i in range(1, n_iter):
        beta_proposal = beta_current + np.random.normal(0, proposal_sd)
        log_post_proposal = log_posterior(beta_proposal, X, y, task_ids)
        log_accept_ratio = log_post_proposal - log_post_current

        if np.log(np.random.rand()) < log_accept_ratio:
            beta_current = beta_proposal
            log_post_current = log_post_proposal

        samples[i] = beta_current

    return samples[burn_in:]

# Run sampler and summarize
posterior_samples = metropolis_sampler(X, y, groups)

posterior_summary = pd.DataFrame({
    "Posterior Mean": posterior_samples.mean(axis=0),
    "Posterior Std": posterior_samples.std(axis=0),
    "95% CI Lower": np.percentile(posterior_samples, 2.5, axis=0),
    "95% CI Upper": np.percentile(posterior_samples, 97.5, axis=0)
}, index=["brand_N", "brand_P", "ad_yes", "price"])

::::


In [ ]:
# | echo: false
posterior_summary

The posterior means are:

- **brand_N**: 0.95  
- **brand_P**: 0.50  
- **ad_yes**: -0.74  
- **price**: -0.100  

Posterior standard deviations closely match the MLE standard errors (within ±0.001). The 95% credible intervals are nearly identical to the MLE confidence intervals. These results reflect strong agreement between the two methods given the large sample size and weakly informative priors.


In [ ]:
#| echo: false
#| fig-cap: "Posterior Histogram for brand_N"
#| fig-align: "center"

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

param_index = 0
param_name = "brand_N"
samples = posterior_samples[:, param_index]

plt.figure(figsize=(6, 4))
sns.histplot(samples, bins=30, kde=True, stat="density", color="steelblue", alpha=0.7)
plt.axvline(np.mean(samples), color='red', linestyle='--', linewidth=1)
plt.title("Posterior Histogram: brand_N")
plt.xlabel("Value")
plt.ylabel("Density")
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

The posterior histogram is approximately normal and centered near 0.95. The distribution is symmetric and unimodal, with most density between 0.75 and 1.15. This indicates a well-identified parameter with low uncertainty.


In [ ]:
#| echo: false
#| fig-cap: "Trace Plot for brand_N"
#| fig-align: "center"

plt.figure(figsize=(6, 4))
plt.plot(samples, color="steelblue", linewidth=0.6)
plt.axhline(np.mean(samples), color='red', linestyle='--', linewidth=1)
plt.title("Trace Plot for brand_N")
plt.xlabel("Iteration")
plt.ylabel("Sampled Value")
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

The trace plot shows consistent sampling around a stable mean with no visible drift or trends. The values fluctuate tightly across the 10,000 post-burn-in iterations, suggesting good mixing and convergence of the Metropolis-Hastings sampler.

::::{.callout-note collapse="true"}
## Simulating the Data


In [ ]:
bayes_summary = posterior_summary.copy()
bayes_summary.columns = ["Bayes Mean", "Bayes Std. Dev.", "Bayes CI Lower", "Bayes CI Upper"]

mle_summary.columns = ["MLE Mean", "MLE Std. Error", "MLE CI Lower", "MLE CI Upper"]

comparison_df = pd.concat([bayes_summary, mle_summary], axis=1)

:::: 


In [ ]:
# | echo: false
comparison_df

The Bayesian posterior means closely align with the MLE point estimates across all four parameters. Differences are minimal:

- **brand_N**: Posterior mean is 0.95; MLE is 0.94. Both intervals fully overlap.
- **brand_P**: Posterior mean is 0.51 vs. MLE of 0.50.
- **ad_yes**: Both methods estimate the effect around -0.73, with nearly identical uncertainty.
- **price**: Posterior mean is -0.0997; MLE is -0.0995, with near-identical intervals.

Posterior standard deviations match MLE standard errors within 0.002 across all parameters. Given the large sample and weak priors, the Bayesian and MLE results are functionally equivalent and reinforce the same conclusions: consumers prefer Netflix and Prime, dislike ads, and are price sensitive.

## 6. Discussion

If this were real data, the parameter estimates would reveal consumers’ underlying preferences.

- **β_Netflix > β_Prime** implies that, all else equal, consumers prefer Netflix to Prime Video. This reflects a higher perceived utility from Netflix’s offering.

- **β_price < 0** is consistent with economic theory: as price increases, utility decreases, making the product less likely to be chosen.

The direction and magnitude of all estimates are reasonable. Consumers dislike ads, show brand preferences, and are price-sensitive—patterns expected in digital subscription markets.

To simulate or estimate a multi-level MNL model, we must account for individual differences in preferences. Unlike the standard MNL model, which assumes a single set of coefficients shared by all respondents, a hierarchical model allows each respondent to have their own set of parameters.

- **Key changes:**
Instead of using one fixed β, assign each respondent a personal βₖ drawn from a population distribution:  \[ \beta_i \sim \mathcal{N}(\mu, \Sigma) \]

- **Estimation:**  
  Estimate both the respondent-level βᵢs and the population-level parameters (μ, Σ). This requires hierarchical modeling techniques, often implemented via Bayesian MCMC or mixed logit estimation.

This approach better reflects real-world data by capturing preference heterogeneity across individuals.
